In [1]:
import os
import json
import pandas as pd
from collections import defaultdict
from rapidfuzz import fuzz
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
df = pd.read_csv('../CSVs/8_filtered_genres.csv',index_col=0).drop('index',axis=1)

In [3]:
df.head()

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,year,genres,num final genres
0,The Vanished Birds,Simon Jimenez,124205.0,55.18,6.37,1.95,0.36,1.58,2020.0,"['Science Fiction', 'Fantasy', 'Adult']",3
1,The Price of Honor,Jonathan P. Brazee,77253.0,35.35,8.71,2.63,0.71,1.92,2017.0,['Science Fiction'],1
2,The Case of the Baker Street Irregulars,Anthony Boucher,80557.0,32.33,8.41,3.72,1.64,2.08,1940.0,"['Mystery', 'Crime', 'Classics']",3
3,Wildoak,C. C. Harrington,55602.0,74.34,6.92,3.04,1.16,1.87,2022.0,"['Historical Fiction', 'Young Adult']",2
4,The Holiday,T. M. Logan,101767.0,50.30,8.02,3.06,1.12,1.93,2019.0,"['Thriller', 'Mystery', 'Crime', 'Suspense']",4


In [4]:
def get_word_counts_from_json(author, title):
    file_path = f'../word-counts/{author}/{title}/word-counts.json'
    word_counts = read_json(file_path)
    return word_counts

In [5]:
def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

The formatting of the directory is as follows: Within the 'word-counts' folder, there are about 15,000 folders representing authors. In the folder of each author, there is a subfolder for each of their books. And in each book folder, there is one file: A JSON containing the frequency of each word that appears in the book. 

In [6]:
def process_directory_structure(root_dir):
    authors = set()
    books = defaultdict(set)
    df = pd.DataFrame()


    for author in os.listdir(root_dir):
        if os.path.isdir(f'{root_dir}/{author}'):
            authors.add(author)
            author_dir = os.path.join(root_dir, author)

            for book in os.listdir(author_dir):
                book_dir = os.path.join(author_dir, book)
                books[author].add(book)

    return books

In [7]:
root_dir = '../word-counts/'
books = process_directory_structure(root_dir)

In [8]:
def add_data_to_dataframe(df, author, book, word_counts):
    word_counts['Author'] = author
    word_counts['Book'] = book
    df = pd.concat(word_counts, ignore_index=True)
    return df

In [9]:
def clean(name):
    to_remove = [':',',','’','?','/']
    for char in to_remove:
        name = name.replace(char, '')
    name = name.lower().replace(' ','-').replace('&','and').replace('.-','-').replace('.','-').strip('-')
    return name

In [10]:
def get_wordcounts(orig_title, orig_author, correct_books, threshold=80):
    # Reformat title and author's name to match folder names
    title = clean(orig_title)
    author = clean(orig_author)
    correct_books_copy = correct_books.copy()
    
    
    # Check if the author's name is in the list of folders at the author level
    if author in correct_books_copy.keys():
        # If found, check the author's folder for the title folder. 
        if title in correct_books[author]:
            return get_word_counts_from_json(author,title)
        else: 
            # Check for title names that are close to the cleaned title name
            for correct_title in correct_books[author]:
                if fuzz.ratio(title, correct_title) > threshold:
                    return get_wordcounts(correct_title, author, correct_books, threshold)
        return -1
    else: 
        # If the exact author's name wasn't found, check for extremely similar authors' names. 
        for correct_author in correct_books_copy.keys():
            if fuzz.ratio(author, correct_author) > 95:
                return get_wordcounts(title, correct_author, correct_books, threshold)
        # If that doesn't work, check for moderately similar authors' names. 
        for correct_author in correct_books_copy.keys():
            if fuzz.ratio(author, correct_author) > threshold:
                return get_wordcounts(title, correct_author, correct_books, threshold)
        return -1

In [11]:
books = process_directory_structure(root_dir)

In [12]:
# for i in tqdm(range(len(df))):
#     returned =  get_wordcounts(df.iloc[i]['title'],df.iloc[i]['author'],books,80)
#     if returned == -1:
#         print(f"Did not work for {df.iloc[i]['title']}, by {df.iloc[i]['author']}")

The only book I wasn't able to find was a book whose title in the dataframe includes a subtitle, and whose matching folder does not! An easy fix. 

In [13]:
df.loc[df['author'] == 'Amelia Abraham']

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,year,genres,num final genres
18673,Queer Intentions: A (Personal) journey through...,Amelia Abraham,86726.0,29.31,8.8,3.12,1.13,1.98,2019.0,['Memoir'],1


In [14]:
df.loc[df['author'] == 'Amelia Abraham', 'title'] = "Queer Intentions"

In [15]:
df.loc[df['author'] == 'Amelia Abraham']

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,year,genres,num final genres
18673,Queer Intentions,Amelia Abraham,86726.0,29.31,8.8,3.12,1.13,1.98,2019.0,['Memoir'],1


In [16]:
# Some of the columns have names that could be words in books. Better make each column name two words.
df.columns = ['book title', 'book author', 'total words', 'vividness score', 'passive voice',
       'all adverbs', 'ly-adverbs', 'non-ly-adverbs', 'publication year', 'book genres',
       'num genres']

In [17]:
df.head()

,book title,book author,total words,vividness score,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,publication year,book genres,num genres
0,The Vanished Birds,Simon Jimenez,124205.0,55.18,6.37,1.95,0.36,1.58,2020.0,"['Science Fiction', 'Fantasy', 'Adult']",3
1,The Price of Honor,Jonathan P. Brazee,77253.0,35.35,8.71,2.63,0.71,1.92,2017.0,['Science Fiction'],1
2,The Case of the Baker Street Irregulars,Anthony Boucher,80557.0,32.33,8.41,3.72,1.64,2.08,1940.0,"['Mystery', 'Crime', 'Classics']",3
3,Wildoak,C. C. Harrington,55602.0,74.34,6.92,3.04,1.16,1.87,2022.0,"['Historical Fiction', 'Young Adult']",2
4,The Holiday,T. M. Logan,101767.0,50.30,8.02,3.06,1.12,1.93,2019.0,"['Thriller', 'Mystery', 'Crime', 'Suspense']",4


In [18]:
test_df = df.head()

In [19]:
test_df = test_df.join(test_df.apply(lambda x:  pd.Series(get_wordcounts(x['book title'], x['book author'],books)),axis=1))

In [20]:
test_df

,book title,book author,total words,vividness score,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,publication year,book genres,...,zone,zoned,zoo,zoom,zoomed,zooming,zucar,émigré,émigrés,über
0,The Vanished Birds,Simon Jimenez,124205.0,55.18,6.37,1.95,0.36,1.58,2020.0,"['Science Fiction', 'Fantasy', 'Adult']",...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,The Price of Honor,Jonathan P. Brazee,77253.0,35.35,8.71,2.63,0.71,1.92,2017.0,['Science Fiction'],...,3.0,NaN,NaN,2.0,6.0,NaN,NaN,NaN,NaN,NaN
2,The Case of the Baker Street Irregulars,Anthony Boucher,80557.0,32.33,8.41,3.72,1.64,2.08,1940.0,"['Mystery', 'Crime', 'Classics']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.0,1.0
3,Wildoak,C. C. Harrington,55602.0,74.34,6.92,3.04,1.16,1.87,2022.0,"['Historical Fiction', 'Young Adult']",...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Holiday,T. M. Logan,101767.0,50.30,8.02,3.06,1.12,1.93,2019.0,"['Thriller', 'Mystery', 'Crime', 'Suspense']",...,1.0,NaN,NaN,2.0,7.0,4.0,NaN,NaN,NaN,NaN


In [21]:
len(df)

20490

In [22]:
df_chunks = []
for i in range(20):
    df_chunks.append(df.iloc[1000*i:1000*(i+1)])

In [23]:
df_chunks.append(df.iloc[20000:len(df)])

In [24]:
complete_chunks = []

In [25]:
for chunk in df_chunks[len(complete_chunks):len(df_chunks)]:
    complete_chunks.append(chunk.join(chunk.progress_apply(lambda x: pd.Series(get_wordcounts(x['book title'], x['book author'], books)), axis=1)))

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/490 [00:00<?, ?it/s]

In [26]:
for chunk in complete_chunks: 
    print(chunk.shape)

(1000, 394352)
(1000, 389858)
(1000, 404815)
(1000, 363286)
(1000, 376105)
(1000, 372208)
(1000, 362448)
(1000, 365104)
(1000, 380801)
(1000, 377421)
(1000, 374421)
(1000, 379593)
(1000, 386220)
(1000, 361970)
(1000, 366176)
(1000, 380285)
(1000, 381545)
(1000, 383617)
(1000, 389048)
(1000, 322558)
(490, 214767)


In [28]:
complete_df = pd.concat(complete_chunks)

MemoryError: Unable to allocate 20.5 MiB for an array with shape (2683940,) and data type int64

In [ ]:
for i, chunk in enumerate(complete_chunks[1:]):
    chunk.to_csv(f'../CSVs/wordcount{i}.csv')